<a href="https://colab.research.google.com/github/Rogerio-mack/Machine-Learning-I/blob/main/ML6_CV_GridSearch_ex_solucao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="Universidade Presbiteriana Mackenzie">
</head>

<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->

# **6. Validação Cruzada e *GridSearch***

---



Após fazer os exercícios deste laboratório responda ao **questionário correspondente da aula no Moodle**.



# Caso: **Propensão de Compra de Clientes por Telemarketing** 

https://archive.ics.uci.edu/ml/datasets/Bank+Marketing

Os dados acima estão relacionados com campanhas de marketing direto de uma instituição bancária portuguesa. As campanhas de marketing foram baseadas em telefonemas. Freqüentemente, era necessário mais de um contato para o mesmo cliente, para acessar se o produto (depósito bancário) seria ('yes') ou não ('no') assinado.

Os dados estão na URL: http://meusite.mackenzie.br/rogerio/TIC/bank-full.csv




# Caso: **Classificação de Tipos de Vidro para Reciclagem**

Nossa base de dados classifica vidros industrializados em 7 categorias conforme suas características químicas:

* Classe 1: janelas de construção (processadas com flutuação)
* Classe 2: janelas de construção (processadas sem flutuação)
* Classe 3: janelas do veículo (processadas com flutuação)
* Classe 4: janelas do veículo (processadas sem flutuação)
* Classe 5: recipientes
* Classe 6: talheres
* Classe 7: faróis

(*algumas dessas classes podem não estar presentes no data-set*).

Os dados estão na URL: https://github.com/Rogerio-mack/Machine-Learning-I/raw/main/data/glasses.csv

Aqui vai nos interessar classificar os vidros para efeito de reciclagem em 3 categorias:

* **C = Vidros de Construção**
* **V = Vidros de Veículos**
* **O = Outros**

E para isso vamos empregar uma seleção de hiperparâmetros de modelos com o GridSearch que você aprendeu na aula teórica.




# Exercício. Acesse e Explore os dados 

Qual o atributo classe e quantos casos de cada classe? 

In [ ]:
# Seu código aqui
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
# Seu código aqui
df = pd.read_csv('https://github.com/Rogerio-mack/IA_2022_1S/raw/main/glasses.csv')
df.head()

,Id number,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type of glass
0,1,1.52101,13.64,D,1.10,71.78,0.06,8.75,0.0,0.0,1
1,2,1.51761,13.89,D,1.36,72.73,0.48,7.83,0.0,0.0,1
2,3,1.51618,13.53,C,1.54,72.99,0.39,7.78,0.0,0.0,1
3,4,1.51766,13.21,D,1.29,72.61,0.57,8.22,0.0,0.0,1
4,5,1.51742,13.27,D,1.24,73.08,0.55,8.07,0.0,0.0,1


In [ ]:
df['Type of glass'].value_counts()

2    76
1    70
7    29
3    17
5    13
Name: Type of glass, dtype: int64

In [ ]:
df['Mg'].unique()

array(['D', 'C', 'B', 'A'], dtype=object)

# Exercício. Preparando os dados 

Atribua os valores `X` e `y` que serão empregados para construção dos modelos.

Aqui um pequeno checklist...

1. Verifique a presença de valores nulos, se houverem exclua os registros com valores nulos
2. Note que antes de prosseguir você precisa criar um atributo com as classes `C`, `V` e `O` de reciclagem ou alterar o atributo classe atual.
3. Parece haver necessidade de encode dos dados. Empregue o hot-encode.
4. Há a necessidade de exclusão de atributos para o treinamento? Você vai empregar todos os atributos cabíveis para a tarefa
5. Os dados precisam ser normalizados? Empregue o `StandardScaler`
 

**1. Verifique a presença de valores nulos, se houverem exclua os registros com valores nulos.**

In [ ]:
# Seu código aqui
df.isnull().sum()

Id number        0
RI               0
Na               0
Mg               0
Al               0
Si               0
K                0
Ca               0
Ba               0
Fe               0
Type of glass    0
dtype: int64

**2. Note que antes de prosseguir você precisa criar um atributo com as classes `C`, `V` e `O` de reciclagem ou alterar o atributo classe atual.**

In [ ]:
# Seu código aqui
df['Type of glass'] = df['Type of glass'].replace(1,'C')
df['Type of glass'] = df['Type of glass'].replace(2,'C')
df['Type of glass'] = df['Type of glass'].replace(3,'V')
df['Type of glass'] = df['Type of glass'].replace(4,'V')
df['Type of glass'] = df['Type of glass'].replace(5,'O')
df['Type of glass'] = df['Type of glass'].replace(6,'O')
df['Type of glass'] = df['Type of glass'].replace(7,'O')

df['Type of glass'].value_counts()


C    146
O     42
V     17
Name: Type of glass, dtype: int64

**3. Parece haver necessidade de encode dos dados. Empregue o hot-encode.**

Se necessário, pesquise aqui a função `OneHotEncoder` do scikit-learn o a função `pd.get_dummies` do Pandas.

In [ ]:
# Solução 1
from sklearn.preprocessing import OneHotEncoder

hotencode = OneHotEncoder()
hotencode.fit(df[['Mg']])

hot_df = pd.DataFrame(hotencode.transform(df[['Mg']]).toarray(),columns=list(hotencode.categories_[0]))
df1 = pd.concat([df, hot_df],axis=1)
df1.head()

,Id number,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type of glass,A,B,C,D
0,1,1.52101,13.64,D,1.10,71.78,0.06,8.75,0.0,0.0,C,0.0,0.0,0.0,1.0
1,2,1.51761,13.89,D,1.36,72.73,0.48,7.83,0.0,0.0,C,0.0,0.0,0.0,1.0
2,3,1.51618,13.53,C,1.54,72.99,0.39,7.78,0.0,0.0,C,0.0,0.0,1.0,0.0
3,4,1.51766,13.21,D,1.29,72.61,0.57,8.22,0.0,0.0,C,0.0,0.0,0.0,1.0
4,5,1.51742,13.27,D,1.24,73.08,0.55,8.07,0.0,0.0,C,0.0,0.0,0.0,1.0


In [ ]:
# Solução 2
df2 = pd.concat([df, pd.get_dummies(df[['Mg']])],axis=1)
df2.head()

,Id number,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type of glass,Mg_A,Mg_B,Mg_C,Mg_D
0,1,1.52101,13.64,D,1.10,71.78,0.06,8.75,0.0,0.0,C,0,0,0,1
1,2,1.51761,13.89,D,1.36,72.73,0.48,7.83,0.0,0.0,C,0,0,0,1
2,3,1.51618,13.53,C,1.54,72.99,0.39,7.78,0.0,0.0,C,0,0,1,0
3,4,1.51766,13.21,D,1.29,72.61,0.57,8.22,0.0,0.0,C,0,0,0,1
4,5,1.51742,13.27,D,1.24,73.08,0.55,8.07,0.0,0.0,C,0,0,0,1


In [ ]:
# Usando aqui a solução 2 (mais simples e suficiente aqui)

df = df2

**4 Há a necessidade de exclusão de atributos para o treinamento? Você vai empregar todos os atributos cabíveis para a tarefa**

In [ ]:
# Seu código aqui
X = df.drop(columns=['Type of glass','Id number','Mg'])
y = df['Type of glass']

**5. Os dados precisam ser normalizados? Empregue o `StandardScaler`**

In [ ]:
# Seu código aqui
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X)

X = scaler.transform(X) 

In [ ]:
print( X[0], len(X[0]) )

0.8603299530312135 12


# Exercício. Treinando o Modelo

Você vai treinar um modelo de Árvore de Decisão buscando os melhores hiperparâmetros de 'max_depth' e 'criterion' (pesquise os possíveis valores na documentação do scikit-learn). Entretanto, no lugar da acuracidade, você empregar o F1 score (`f1_macro`) que é uma métrica que balanceia os resultados de precisão e recall. Você pode empregar como modelo os códigos do notebook de teoria.

Aqui um checklist do que precisa ser feito...

1. Separe os dados de Treinamento e Teste 
> Empregue 0.3 dos dados para teste, estratificados e não deixe de empregar o seed 123.

2. Defina uma DecisionTree como Estimador Base.
> Não deixe de empregar o parâmetro `random_state=123` no estimador base para a reprodutibilidade dos resultados. 

3. Especifique o range dos valores 'max_depth' e 'criterion' que você deseja empregar
> Você pode ter que pesquisar isso na documentação do scikit-learn.

4. Configure o `GridSearchCV`
> Você deve empregar 5 partições e empregar o score de `f1_macro` para a seleção dos melhores hiperparâmetros.

5. Verifique os Resultados
> Verifique os hiperparâmetros selecionados e gere um classification report para ver as métricas do modelo.


**Nota**: você pode ignorar FitFailedWarning que podem ocorrer ao longo do treinamento. Esse warning é mesmo esperado aqui.

In [ ]:
# seu código aqui 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

clf = DecisionTreeClassifier(max_depth=3, criterion='entropy')
clf.get_params

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=123)

base_estimator = DecisionTreeClassifier(random_state=123)
param_grid = {'max_depth': range(3,10), 'criterion': ['gini','entropy','log_loss']}

clf = GridSearchCV(base_estimator, param_grid, cv=5, scoring='f1_macro')
clf.fit(X_train, y_train)

# print(clf.cv_results_)
print(clf.best_estimator_)

print()
print("Detailed classification report:")
print()
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
print()

DecisionTreeClassifier(criterion='entropy', max_depth=7, random_state=123)

Detailed classification report:

              precision    recall  f1-score   support

           C       0.91      0.95      0.93        44
           O       0.87      1.00      0.93        13
           V       1.00      0.20      0.33         5

    accuracy                           0.90        62
   macro avg       0.93      0.72      0.73        62
weighted avg       0.91      0.90      0.88        62




/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
35 fits failed out of a total of 105.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
35 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 942, in fit
    X_idx_sorted=X_idx_sorted,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 352, in fit
    criterion = CRITERIA_CLF[self.criterion](
KeyError: 'log_loss'

  warnings.warn(some_fits_failed_mes